# **ETL Unificado**  
#### **En este notebook se unificaran las reviews y los locales de Yelp y Google Maps en dos datasets:**  

**reviews_unificadas.parquet (con las reviews de ambas fuentes) y restaurantes_unificados.parquet (con los locales de ambas fuentes)**  

Importamos librerias necesarias:

In [1]:
import pandas as pd

## **Extracción de Datos:**  
Importamos los parquets necesarios:

In [2]:
# Google Maps:
reviews_g = pd.read_parquet("../../Data/data_procesada/reviews_google_maps.parquet")
restaurantes_g = pd.read_parquet("../../Data/data_procesada/restaurantes.parquet")

# Yelp:
reviews_y = pd.read_parquet("../../Data/data_procesada/Dataset_Review_simplificado.parquet")
restaurantes_y = pd.read_parquet("../../Data/data_procesada/Dataset_Business_simplificado.parquet")

## **Transformación de Datos:**  

Visualizamos los dataframes empezando primero por los de reviews y después los de locales.  
Reviews:

In [3]:
reviews_g.head(1)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
0,109434968607034492648,Tony Pinto,2013,11,18,14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


In [4]:
reviews_y.head(1)

,user_id,stars_Dataset_Review,text,id_review,id_user,id_business,anho,mes,dia,hora,name,city,state,stars_Dataset_Business,review_count,yelp_id
0,mh_-eMZ6K5RLWhZyISBhwA,3.0,"If you decide to eat here, just be aware it is...",0,124234,1867,2018,07,07,22:09:11,Turning Point of North Wales,North Wales,Pennsylvania,3.0,169,mh_-eMZ6K5RLWhZyISBhwA


Locales (restaurantes):

In [5]:
restaurantes_g.head(1)

,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.29213,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant


In [6]:
restaurantes_y.head(1)

,id_business,name,city,state,postal_code,latitude,longitude,stars,review_count,hours_Dataset_Business,first_category
6,3,St Honore Pastries,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,"{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '...",Restaurants


### **Trabajaremos primero con ambos dataframes de reviews.**  
Transformaremos y eliminaremos varias columnas con el objetivo de unirlas en un solo dataframe. 

En primer lugar trabajaremos de manera segura creando copias de ambos dataframes:

In [11]:
df_review_g = reviews_g.copy()

df_review_y = reviews_y.copy()

#### **Parte de reviews de Yelp**

En el dataframe de reviews de Yelp hay columnas sobrantes que serán eliminadas:

In [12]:
drop = ["user_id", "id_review", "stars_Dataset_Business", "review_count", "yelp_id"]

df_review_y.drop(drop, axis=1, inplace=True)

Ahora crearemos la columna 'user_name' uniendo la palabra 'user' más la columna 'id_user' ('user' + 'id_user'). Esto se hace para tener un nombre de usuario único para las reviews de Yelp asi como lo tienen las reviews de Google Maps. 

In [13]:
df_review_y['user_name'] = 'user' + df_review_y['id_user'].astype(str)

Ahora vamos a renombrar las columnas y cambiar su orden para hacerlas lo más parecidas al dataframe de Google Maps:

In [14]:
# Renombramos columnas:

df_review_y = df_review_y.rename(columns={
    "stars_Dataset_Review": "rating", 
    "id_user": "user_id",
    "anho": "year",
    "mes": "month",
    "dia": "day",
    "hora": "hour",
    "name": "restaurant_name",
    "id_business": "restaurant_id"

})

# Reordenamos las columnas:
orden = ["user_id", "user_name", "year", "month", "day", "hour", "rating", "text", "restaurant_name", "state", "city", "restaurant_id"]

df_review_y = df_review_y[orden]
df_review_y.head(1)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,restaurant_id
0,124234,user124234,2018,07,07,22:09:11,3.0,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,Pennsylvania,North Wales,1867


Transformamos la columna 'rating' de Yelp, esta en float y la pasaremos a int:

In [15]:
df_review_y["rating"] = df_review_y["rating"].astype(int)

También modificaremos las columnas 'month' y 'day' ya que en las fechas de un digito tienen un cero por delante (se puede observar arriba en el dataframe):

In [16]:
# Eliminar ceros iniciales de las columnas 'month' y 'day'
df_review_y['month'] = df_review_y['month'].astype(str).str.lstrip('0')
df_review_y['day'] = df_review_y['day'].astype(str).str.lstrip('0')

# Convertir las columnas a tipo int
df_review_y['month'] = df_review_y['month'].astype(int)
df_review_y['day'] = df_review_y['day'].astype(int)

# Visualizamos:
df_review_y.head(1)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,restaurant_id
0,124234,user124234,2018,7,7,22:09:11,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,Pennsylvania,North Wales,1867


Ahora modificaremos los tipos de datos del dataframe:

In [17]:
df_review_y['user_id'] = df_review_y['user_id'].astype(str)
df_review_y['year'] = df_review_y['year'].astype(int)
df_review_y["restaurant_id"] = df_review_y["restaurant_id"].astype(str)

#### **Parte de reviews de Google Maps**

Para hacerlo igual al dataframe de Yelp cambiaremos el nombre de la columna 'gmap_id' por 'restaurant_id'

In [18]:
df_review_g = df_review_g.rename(columns={"gmap_id": "restaurant_id"})

df_review_g.head(1)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,restaurant_id
0,109434968607034492648,Tony Pinto,2013,11,18,14:03:54.547,5,I came by yesterday to pick up some pizza on t...,Raffaello Kosher Pizza,New_York,New York,0x89c258ffaeaba947:0x8355860772a595a9


#### **Concatenar Reviews**

Ahora concatenaremos las reviews de Yelp y Google Maps en un solo dataframe:

In [19]:
df_reviews_combined = pd.concat([df_review_g, df_review_y], ignore_index=True)

In [20]:
df_reviews_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544121 entries, 0 to 2544120
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          object
 1   user_name        object
 2   year             int32 
 3   month            int32 
 4   day              int32 
 5   hour             object
 6   rating           int64 
 7   text             object
 8   restaurant_name  object
 9   state            object
 10  city             object
 11  restaurant_id    object
dtypes: int32(3), int64(1), object(8)
memory usage: 203.8+ MB


### **Ahora trabajaremos con los datos de restaurantes.**  
Transformaremos y eliminaremos varias columnas con el objetivo de unirlas en un solo dataframe. 

En primer lugar trabajaremos de manera segura creando copias de ambos dataframes:

In [21]:
restaurantes_g_copy = restaurantes_g.copy()

restaurantes_y_copy = restaurantes_y.copy()

#### **Parte de restaurantes de Yelp**

En el dataframe de restaurantes de Yelp hay una columna sobrante que será eliminada ('hours_Dataset_Business'):

In [22]:
restaurantes_y_copy.drop("hours_Dataset_Business", axis=1, inplace=True)

Añadiremos la columna de 'price_numeric' que se encuentra también en el dataframe de Yelp.   
(Aunque en el caso de Yelp al no tener un dato todos sus valores serán 0, equivalente a 'Sin Dato')

In [23]:
restaurantes_y_copy["price_numeric"] = 0

Cambiaremos el tipo de dato de la columna 'id_business' y renombraremos las columnas:

In [24]:
# Cambiamos el tipo de dato de int a str:
restaurantes_y_copy["id_business"] = restaurantes_y_copy["id_business"].astype(str)

# Renombramos columnas:
restaurantes_y_copy = restaurantes_y_copy.rename(columns={
    "id_business": "restaurant_id", 
    "first_category": "categories"
})

restaurantes_y_copy.head(1)

,restaurant_id,name,city,state,postal_code,latitude,longitude,stars,review_count,categories,price_numeric
6,3,St Honore Pastries,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,Restaurants,0


Nuevo orden de columnas:

In [25]:
# Reordenamos las columnas:
orden_rest_y = ["name", "state", "city", "postal_code", "latitude", "longitude", "stars", "review_count", "price_numeric", "categories", "restaurant_id"]

restaurantes_y_copy = restaurantes_y_copy[orden_rest_y]
restaurantes_y_copy.head(1)

,name,state,city,postal_code,latitude,longitude,stars,review_count,price_numeric,categories,restaurant_id
6,St Honore Pastries,Pennsylvania,Philadelphia,19107,39.955505,-75.155564,4.0,80,0,Restaurants,3


#### **Parte de restaurantes de Google Maps**

En el dataframe de restaurantes de Google Maps hay una columna sobrante que será eliminada ('address'):

In [26]:
restaurantes_g_copy.drop("address", axis=1, inplace=True)

Renombramos columnas y reordenamos el dataframe:

In [27]:
# Renombramos columnas:
restaurantes_g_copy = restaurantes_g_copy.rename(columns={
    "avg_rating": "stars", 
    "num_of_reviews": "review_count",
    "category": "categories",
    "gmap_id": "restaurant_id"
})

# Reordenamos las columnas:
orden_rest_g = ["name", "state", "city", "postal_code", "latitude", "longitude", "stars", "review_count", "price_numeric", "categories", "restaurant_id"]

restaurantes_g_copy = restaurantes_g_copy[orden_rest_g]
restaurantes_g_copy.head(1)

,name,state,city,postal_code,latitude,longitude,stars,review_count,price_numeric,categories,restaurant_id
0,San Soo Dang,California,Los Angeles,90005,34.058092,-118.29213,4.4,18,0,Korean restaurant,0x80c2c778e3b73d33:0xbdc58662a4a97d49


#### **Concatenar Restaurantes**

Ahora concatenaremos los restaurantes de Yelp y Google Maps en un solo dataframe:

In [28]:
df_restaurantes_combined = pd.concat([restaurantes_g_copy, restaurantes_y_copy], ignore_index=True)

In [29]:
df_restaurantes_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31761 entries, 0 to 31760
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           31761 non-null  object 
 1   state          31761 non-null  object 
 2   city           31761 non-null  object 
 3   postal_code    31761 non-null  int64  
 4   latitude       31761 non-null  float64
 5   longitude      31761 non-null  float64
 6   stars          31761 non-null  float64
 7   review_count   31761 non-null  int64  
 8   price_numeric  31761 non-null  int64  
 9   categories     31761 non-null  object 
 10  restaurant_id  31761 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 2.7+ MB


## **Carga de Datos:**  

Guardaremos nuestros dataframes en archivos parquet:

Datos de las reviews, Dataframe 'df_reviews_combined'. Lo usaremos para análisis.

In [31]:
df_reviews_combined.to_parquet("../../Data/data_procesada/reviews_unificadas.parquet")

Datos de las restaurantes, Dataframe 'df_restaurantes_combined'. Lo usaremos para análisis.

In [32]:
df_restaurantes_combined.to_parquet("../../Data/data_procesada/restaurantes_unificados.parquet")